In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import time
import regex

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/RMP/scraped_comments.csv')

In [5]:
df.head(5)

,comment_id,firstName,lastName,prof_class,comment,ratingTags,date,attendanceMandatory,grade,clarityRating,difficultyRating,helpfulRating,textbookUse,thumbsDownTotal,thumbsUpTotal,wouldTakeAgain
0,UmF0aW5nLTIyODAzODMy,Marty,Beans,FINITEMATH,Very nice and understanding. A lot of homework...,NaN,2014-01-29 16:17:28 +0000 UTC,Y,B,4,3,4,5.0,0,0,NaN
1,UmF0aW5nLTE5NjU2ODY4,Marty,Beans,MTH106,She is very helpful. Gives EC if you go to tut...,NaN,2012-01-03 02:19:53 +0000 UTC,NaN,NaN,4,2,4,5.0,0,0,NaN
2,UmF0aW5nLTEyMTIwMDcz,Marty,Beans,MATHSUMFIN,she was nice. good job her,NaN,2006-07-31 20:31:38 +0000 UTC,NaN,NaN,4,4,5,NaN,0,0,NaN
3,UmF0aW5nLTExMTc4MTMx,Marty,Beans,DEVELOPMATH,Professor Beans is one of the best Math teache...,NaN,2005-12-15 23:07:38 +0000 UTC,NaN,NaN,5,1,5,NaN,0,0,NaN
4,UmF0aW5nLTEwMTkzMTcx,Marty,Beans,MATH100,Big smile. Big goals. Nice lady. Bright and ch...,NaN,2005-08-31 21:17:16 +0000 UTC,N,A-,5,1,5,NaN,0,0,NaN


In [6]:
print(df.shape)
df['comment_id'].value_counts() # Mistakes with scraping that led to duplicates

(4121664, 16)


UmF0aW5nLTMxOTczNDg=    2
UmF0aW5nLTExNTY0NzQ4    2
UmF0aW5nLTMxNjU1MTc0    2
UmF0aW5nLTEwOTYyODIz    2
UmF0aW5nLTM3OTkwNzk=    2
                       ..
UmF0aW5nLTIzMjgwMjQx    1
UmF0aW5nLTEyMDYwMzQ4    1
UmF0aW5nLTM2OTY4NTA=    1
UmF0aW5nLTI2MjI2MzYw    1
UmF0aW5nLTQxNTc5Ng==    1
Name: comment_id, Length: 4121613, dtype: int64

In [7]:
# Dropping duplicates
print("Size before dropping duplicates:", df.shape)
df.drop_duplicates(subset=['comment_id'], inplace=True)
print("Size after dropping duplicates:", df.shape)
df['comment_id'].value_counts()

Size before dropping duplicates: (4121664, 16)
Size after dropping duplicates: (4121613, 16)


UmF0aW5nLTIyODAzODMy    1
UmF0aW5nLTIwNzgxOTI4    1
UmF0aW5nLTIzNTYzMDky    1
UmF0aW5nLTIzNDk5MzY1    1
UmF0aW5nLTIzMzYxNTA3    1
                       ..
UmF0aW5nLTMwMzQ5NzU=    1
UmF0aW5nLTE2MDUwNzM1    1
UmF0aW5nLTE1NDQ5ODM2    1
UmF0aW5nLTE3NzcwODQ=    1
UmF0aW5nLTQxNTc5Ng==    1
Name: comment_id, Length: 4121613, dtype: int64

In [8]:
print("Number of null ratings:", df['clarityRating'].isna().sum()) # No missing ratings
df['clarityRating'].value_counts()

Number of null ratings: 0


 5    1686243
 4     835475
 1     639677
 3     508522
 2     451695
-1          1
Name: clarityRating, dtype: int64

In [9]:
df = df[df['clarityRating'] != -1]
df['clarityRating'].value_counts()

5    1686243
4     835475
1     639677
3     508522
2     451695
Name: clarityRating, dtype: int64

In [10]:
print("Rows with empty comments:", df['comment'].isna().sum()) # Empty comments exist
print("Rows named \'No Comments\':", (df['comment'] == 'No Comments').sum())

Rows with empty comments: 7310
Rows named 'No Comments': 184430


In [11]:
df.dropna(subset=['comment'], inplace=True)
df = df[df['comment'] != 'No Comments']

In [12]:
print("Rows with empty comments:", df['comment'].isna().sum())
print("Rows named \'No Comments\':", (df['comment'] == 'No Comments').sum())

Rows with empty comments: 0
Rows named 'No Comments': 0


In [13]:
df.reset_index(drop=True, inplace=True)

# Swear words
We might want to get a count for how often these appear, or their word counts in the entire population

In [14]:

df_swear_comments = df[df['comment'].str.contains("(fuck|shit|asshole|dick|penis|cock|vagina|pussy|cunt|retard|bitch|slut|whore)")]
df_swear_comments.reset_index(drop=True, inplace=True)
print(df_swear_comments.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


(6371, 16)


In [15]:
df_swear_comments.iloc[15:20]

,comment_id,firstName,lastName,prof_class,comment,ratingTags,date,attendanceMandatory,grade,clarityRating,difficultyRating,helpfulRating,textbookUse,thumbsDownTotal,thumbsUpTotal,wouldTakeAgain
15,UmF0aW5nLTEwNTY4MTY1,Barb,Stengel,EDFN111,she's insane.\r\n better than the other retard...,NaN,2005-10-09 13:42:34 +0000 UTC,NaN,NaN,4,1,5,NaN,0,0,NaN
16,UmF0aW5nLTM0NjM2ODY=,Barb,Stengel,EDFN211,"Excellent, ten times better than all the other...",NaN,2005-03-16 14:10:04 +0000 UTC,NaN,NaN,5,1,5,NaN,0,0,NaN
17,UmF0aW5nLTEyMzg3ODA3,Joseph,Caspar,MAT110,U know the scene from Saw where the guy cuts o...,NaN,2006-11-09 15:10:56 +0000 UTC,NaN,NaN,1,5,1,5.0,0,0,NaN
18,UmF0aW5nLTExNjQyMTc4,John,Kim,COMPLIT,"This guy is pretty cocky, but his intelligence...",NaN,2006-04-01 02:58:02 +0000 UTC,NaN,NaN,5,1,3,NaN,0,0,NaN
19,UmF0aW5nLTEwOTU0MDA4,Bessma,Momani,HIST130,I had her last year. She was a great prof. Fun...,NaN,2005-11-19 16:08:46 +0000 UTC,Y,Fail,5,2,5,NaN,0,0,NaN


In [16]:
import html
list_of_comments_with_swears = list(df_swear_comments['comment'])
for comment in list_of_comments_with_swears[15:20]:
  print(html.unescape(comment) + '\n')


she's insane.
 better than the other retards over at stayer.

Excellent, ten times better than all the other Stayer retards

U know the scene from Saw where the guy cuts out his own eye? It's kinda like that. If you get him...kill yourself. He's a living corpse who relies on ur stress for sustenance.  Every day he reminds the class they are basicly retarded, and yells at u if u don't understand the lesson.  If u can, kill someone so u can get their spot in another class.

This guy is pretty cocky, but his intelligence is the key to getting a good grade! I did not buy any of the books he told us to.. I just went to every lecture and took notes on his reviews. I got a decent grade... As for future students, buy the books and read the assignments.. you have to finish them in like a week.. not much time, but for an A, y

I had her last year. She was a great prof. Funny, good lectures, puts notes on reserve at library, and her test are good. If anyone doesn't like her i imagine she made fun

In [17]:
df_swear_comments['clarityRating'].value_counts()

5    1695
1    1658
4    1077
2     999
3     942
Name: clarityRating, dtype: int64

In [26]:
print(df.shape)
df_special_comments = df[df['comment'].str.contains("[*#@%$=_&]+")]
df_special_comments.reset_index(drop=True, inplace=True)
print(df_special_comments.shape)

(3929872, 16)
(388558, 16)


In [36]:
df_html_entities = df[df['comment'].str.contains('&([a-zA-z]+|#\d+);')]
print(df_html_entities.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


(180667, 16)


In [47]:
import re
html_entities = set()
for comment in df_html_entities['comment']:
  all_entities = set(re.findall('(&([a-zA-z]+|#\d+);)', comment))
  for entity in all_entities:
    html_entities.add(entity[0])
html_entities

{'&#1040;',
 '&#1041;',
 '&#1042;',
 '&#1043;',
 '&#1044;',
 '&#1045;',
 '&#1047;',
 '&#1048;',
 '&#1050;',
 '&#1051;',
 '&#1052;',
 '&#1053;',
 '&#1054;',
 '&#1055;',
 '&#1056;',
 '&#1057;',
 '&#1058;',
 '&#1059;',
 '&#1064;',
 '&#1071;',
 '&#1072;',
 '&#1074;',
 '&#1077;',
 '&#1079;',
 '&#1080;',
 '&#1081;',
 '&#1082;',
 '&#1083;',
 '&#1084;',
 '&#1085;',
 '&#1086;',
 '&#1087;',
 '&#1088;',
 '&#1089;',
 '&#1090;',
 '&#1091;',
 '&#1095;',
 '&#1099;',
 '&#1100;',
 '&#1103;',
 '&#12288;',
 '&#12289;',
 '&#12290;',
 '&#12354;',
 '&#12356;',
 '&#12358;',
 '&#12360;',
 '&#12362;',
 '&#12364;',
 '&#12372;',
 '&#12374;',
 '&#12375;',
 '&#12377;',
 '&#12379;',
 '&#12383;',
 '&#12391;',
 '&#12392;',
 '&#12414;',
 '&#12424;',
 '&#12426;',
 '&#12428;',
 '&#12435;',
 '&#12484;',
 '&#12488;',
 '&#12522;',
 '&#12631;',
 '&#12636;',
 '&#128516;',
 '&#146;',
 '&#1488;',
 '&#1491;',
 '&#1492;',
 '&#1493;',
 '&#1497;',
 '&#1499;',
 '&#1500;',
 '&#1501;',
 '&#1502;',
 '&#1510;',
 '&#1512;',
 '&#1514;',


In [19]:
bfp = df[(df['firstName'] == 'Biff')]
bfp[bfp['comment'].str.contains('Very funny')] # On the website the rating is 4.5, but clarityRating is 4 instead

,comment_id,firstName,lastName,prof_class,comment,ratingTags,date,attendanceMandatory,grade,clarityRating,difficultyRating,helpfulRating,textbookUse,thumbsDownTotal,thumbsUpTotal,wouldTakeAgain
4043,UmF0aW5nLTgyNTczMg==,Biff,Pietro,PRECALC,"Very funny, loves what he does and will help y...",NaN,2003-04-12 03:41:26 +0000 UTC,NaN,NaN,4,4,5,NaN,0,0,NaN


# Observations
* There are duplicates in the comments, easy to drop though
* Professor id was not scraped, **NEEDS TO BE DONE IN FUTURE SCRAPES**
* Clarity rating rounds down
* Clarity ratings skew positve, then heavily negative (5, 4, then 1)
* Some comments are null, and some are null with the text "No Comments".
* There are swear words in some of the reviews, though they may not have been admin reviewed. **Grab this feature when scraping**

# TODO


* Have to figure out a way to find reviews that contain words that our models don't have (slang words)
* How to deal with numbers (0-9) and normalize them
* **Deal with html entities, either by parsing them or removing them**